<a href="https://colab.research.google.com/github/pizzab0y/im4talkin/blob/main/references/SeamlessM4T_barbarous_panda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/seamless_communication

Cloning into 'seamless_communication'...
remote: Enumerating objects: 658, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 658 (delta 54), reused 53 (delta 36), pack-reused 542
Receiving objects: 100% (658/658), 546.89 KiB | 5.36 MiB/s, done.
Resolving deltas: 100% (311/311), done.


In [ ]:
cd seamless_communication

/content/seamless_communication


In [ ]:
!pip install -e .

Obtaining file:///content/seamless_communication
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install ffmpeg-python

In [ ]:
# Blatantly copied from this notebook: https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb

"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
from scipy.io.wavfile import write as wav_write
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

# Record audio

In [ ]:
audio, sr = get_audio()
wav_write('input.wav', sr, audio)

Resample audio to 16000 Hz otherwise seamlessm4t won't be able to make sense of it

In [ ]:
import torchaudio
resample_rate = 16000
waveform, sample_rate = torchaudio.load('input.wav')
resampler = torchaudio.transforms.Resample(sample_rate, resample_rate, dtype=waveform.dtype)
resampled_waveform = resampler(waveform)
torchaudio.save('inputre.wav', resampled_waveform, resample_rate)

This will be our input to the model

In [ ]:
display(Audio('inputre.wav'))

# Speech-to-speech translation (S2ST)

In [ ]:
!m4t_predict inputre.wav s2st eng --output_path output.wav --model_name seamlessM4T_large

2023-08-23 08:16:42,205 INFO -- m4t_scripts.predict.predict: Running inference on the GPU.
100% 10.7G/10.7G [00:59<00:00, 194MB/s]
100% 4.93M/4.93M [00:00<00:00, 66.2MB/s]
100% 160M/160M [00:00<00:00, 240MB/s]
2023-08-23 08:18:48,444 INFO -- m4t_scripts.predict.predict: Saving translated audio in eng
2023-08-23 08:18:48,475 INFO -- m4t_scripts.predict.predict: Translated text in eng: This is a test to see if a model can transcribe both Hindi and English. If it can, it would be awesome.


In [ ]:
display(Audio('output.wav'))

# Speech-to-text translation (S2TT)

In [ ]:
!m4t_predict inputre.wav s2tt eng --model_name seamlessM4T_large

2023-08-23 08:18:55,697 INFO -- m4t_scripts.predict.predict: Running inference on the GPU.
Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.
2023-08-23 08:20:10,128 INFO -- m4t_scripts.predict.predict: Translated text in eng: This is a test to see if a model can transcribe both Hindi and English. If it can, it would be awesome.


# Text-to-text translation (T2TT)

In [ ]:
!m4t_predict "This is a test to see if meta's new model can translate this sentence accurately" t2tt hin --src_lang eng

2023-08-23 05:37:23,904 INFO -- m4t_scripts.predict.predict: Running inference on the GPU.
Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.
2023-08-23 05:38:37,748 INFO -- m4t_scripts.predict.predict: Translated text in hin: यह यह देखने के लिए एक परीक्षण है कि क्या मेटा का नया मॉडल इस वाक्य को सटीक रूप से अनुवाद कर सकता है


# Text-to-speech translation (T2ST)

In [ ]:
!m4t_predict "This is a test to see if meta's new model can translate this sentence accurately" t2st hin --src_lang eng --output_path output.wav

2023-08-23 05:42:16,866 INFO -- m4t_scripts.predict.predict: Running inference on the GPU.
Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.
2023-08-23 05:43:36,875 INFO -- m4t_scripts.predict.predict: Saving translated audio in hin
2023-08-23 05:43:36,914 INFO -- m4t_scripts.predict.predict: Translated text in hin: यह यह देखने के लिए एक परीक्षण है कि क्या मेटा का नया मॉडल इस वाक्य को सटीक रूप से अनुवाद कर सकता है


In [ ]:
display(Audio('output.wav'))

[Reference](https://github.com/facebookresearch/seamless_communication/tree/main/scripts/m4t/predict)